In [174]:
%%writefile create_external_db.hql

USE demodb;
DROP TABLE IF EXISTS posts_sample_external;

CREATE EXTERNAL TABLE posts_sample_external (
    id int,
    year string,
    month string

)
ROW FORMAT
    SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
    WITH SERDEPROPERTIES (
        "input.regex" = '^<row(?=.*\\bId="(\\d+)")(?=.*\\bCreationDate="(\\d{4})-(\\d{2})\\S*").*$'
    )
LOCATION '/data/stackexchange1000/posts';

Writing create_external_db.hql


In [175]:
%%writefile create_partitioned_db.hql

SET hive.exec.dynamic.partition=true;
SET hive.exec.dynamic.partition.mode=nonstrict;

USE demodb;

DROP TABLE IF EXISTS posts_sample;

CREATE TABLE posts_sample (
   count int
) PARTITIONED BY (year string, month string);

FROM posts_sample_external
INSERT OVERWRITE TABLE posts_sample
PARTITION (year, month)
SELECT count(*), year, concat(year, '-', month) as month
WHERE year IS NOT NULL
GROUP BY year, concat(year, '-', month);

Writing create_partitioned_db.hql


In [180]:
%%writefile query_3rd_row.hql

USE demodb;

SELECT year, month, count FROM (
    SELECT * FROM (
        SELECT
            year,
            month,
            count,
            ROW_NUMBER() OVER(ORDER BY year, month ASC) as rn
        FROM posts_sample
    ) AS t1
    ORDER BY rn ASC
) AS t2
WHERE rn='3';

Overwriting query_3rd_row.hql


In [177]:
! hive -f create_external_db.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 0.565 seconds
OK
Time taken: 0.719 seconds
OK
Time taken: 0.258 seconds


In [178]:
! hive -f create_partitioned_db.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 0.57 seconds
OK
Time taken: 2.001 seconds
OK
Time taken: 0.277 seconds
Query ID = jovyan_20180519133434_0650f073-19b5-4516-b9b1-6e2438552367
Total jobs = 1
Launching Job 1 out of 1
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1526617807697_0014, Tracking URL = http://d8fa695eb221:8088/proxy/application_1526617807697_0014/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1526617807697_0014
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2018-05-19 13:34:21,928 Stage-1 map = 0%, 

Partition demodb.posts_sample{year=2012, month=2012-05} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2012, month=2012-06} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2012, month=2012-07} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2012, month=2012-08} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2012, month=2012-09} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2012, month=2012-10} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2012, month=2012-11} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2012, month=2012-12} stats: [numFiles=1, numRows=1, totalSize=4, rawDataSize=3]
Partition demodb.posts_sample{year=2013, month=2013-01} stats: [numFiles=1, numR

In [181]:
! hive -f query_3rd_row.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 0.545 seconds
Query ID = jovyan_20180519134545_c2ea49a5-4277-4962-9ef2-8007d5516300
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1526617807697_0016, Tracking URL = http://d8fa695eb221:8088/proxy/application_1526617807697_0016/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1526617807697_0016
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2018-05-19 13:45:20,443 Stage-1 map = 0%,  reduce = 0%
2018-05-19 13:45:25,657 Stage-1 map = 100%, 